In [ ]:
# only run this cell once!
import utils, image_classifier
import numpy as np, os, PIL.Image
import mvnc.mvncapi as mvnc

loaded_ncs = False

# Set params

In [ ]:
img_dir = 'logged_photos0/'
# img_dir = 'my_dogs/'

# Convert `.npy` to `.jpg`

In [ ]:
def convert_from_preprocessed_npy_to_jpg(img_dir):
    for file in os.listdir(img_dir):
        if file[-4:] == '.npy' and ((file[:-4] + '.jpg') not in os.listdir(img_dir)):
            # load and undo effects of preprocessing
            img_arr = np.load(img_dir + file)
            img_arr = (img_arr/np.float32(0.00789)) + np.float32((127.5, 127.5, 127.5))
            img_arr = img_arr.astype(np.uint8)

            # save as regular image
            img = PIL.Image.fromarray(img_arr) # (frame.array - np.float32(mean)) * np.float32(scale)
            img.save(img_dir + file[:-4] + '.jpg')
    
    try:
        return img # returns this if img is defined
    finally:
        return

convert_from_preprocessed_npy_to_jpg(img_dir)

# Test neural net using photos on disk

In [ ]:
def init_ncs(graph_filename='../Models/MobileNet_SSD_caffe/graph'):
    devices = mvnc.enumerate_devices()
    if len(devices) == 0:
        print('No NCS devices found')
        exit()

    # Get a handle to the first enumerated device and open it
    device = mvnc.Device(devices[0])
    device.open()

    # Read the graph file into a buffer
    with open(graph_filename, mode='rb') as f:
        blob = f.read()

    # Load the graph buffer into the NCS
    graph = mvnc.Graph(graph_filename)
    fifo_in, fifo_out = graph.allocate_with_fifos(device, blob)

    return device, graph, fifo_in, fifo_out

In [ ]:
# only get a handle to the device + upload the graph once!
if not loaded_ncs:
    device, graph, fifo_in, fifo_out = init_ncs()
    loaded_ncs = True

In [ ]:
def infer_img(img,
              graph, fifo_in, fifo_out,
              nn_shape=(300,300),
              mean=(127.5, 127.5, 127.5),
              scale=0.00789,
              confidance_threshold=0.7):
    preprocessed_img = (img - np.float32(mean)) * np.float32(scale)

    graph.queue_inference_with_fifo_elem(fifo_in, fifo_out, preprocessed_img, None)
    
    output, userobj = fifo_out.read_elem()

    # Deserialize the output into a python dictionary
    output_dict = utils.deserialize_ssd(output, nn_shape, confidance_threshold)
    
    return output_dict

In [ ]:
label_filename='../Models/MobileNet_SSD_caffe/categories.txt'
labels = [line.rstrip('\n') for line in open(label_filename) if line != 'classes\n']

# class_of_interest = 'all'
# class_of_interest = labels.index('dog')
class_of_interest = labels.index('person')

In [ ]:
imgs = []

for file in os.listdir(img_dir):
    if file[-4:] == '.npy':
        continue
    im = PIL.Image.open(img_dir + file).resize((300,300))
    output_dict = infer_img(np.array(im), graph, fifo_in, fifo_out, confidance_threshold=0.2)
    img = utils.display_image(output_dict, class_of_interest, np.array(im), labels)
    
    imgs.append(PIL.Image.fromarray(img))

len(imgs)

In [ ]:
imgs[0].resize((600,600), PIL.Image.ANTIALIAS)

In [ ]:
imgs[1].resize((600,600), PIL.Image.ANTIALIAS)

In [ ]:
imgs[2].resize((600,600), PIL.Image.ANTIALIAS)

In [ ]:
imgs[3].resize((600,600), PIL.Image.ANTIALIAS)

In [ ]:
imgs[16].resize((600,600), PIL.Image.ANTIALIAS)

# Test aspects of the `picamera`

In [ ]:
import picamera, picamera.array, PIL.Image, time, numpy as np

In [ ]:
cam_shape = (1640, 922)
nn_shape = (320, 304)
with picamera.PiCamera(resolution=cam_shape, framerate=90, sensor_mode=5) as camera:
    time.sleep(1)
    frame = picamera.array.PiRGBArray(camera, size=nn_shape)
    cont_capture = camera.capture_continuous(frame, 'rgb',
                                             resize=nn_shape,
                                             use_video_port=True)# use GPU for resizing
    for i in range(5):        
        frame.seek(0)
        frame.truncate(0)
        t = time.time()
        next(cont_capture)
        print(time.time() - t)
        im = frame.array

print(im.shape)
PIL.Image.fromarray(im)

In [ ]:
t = time.time()
for i in range(10):
    np.array(PIL.Image.fromarray(im).resize((300,300), PIL.Image.ANTIALIAS))
print('avg time: %ims' % ((time.time() - t)/10*1000))